# KMeans Spark

https://spark.apache.org/docs/latest/mllib-clustering.html#k-means

1. Start Spark Cluster:

Link to iPython Notebook: [Spark on Stampede](../02_hadoop_on_hpc/Stampede.ipynb)

In [2]:
from numpy import array
from math import sqrt

%run ../util/init_spark.py
%run ../env.py

print "SPARK HOME: %s"%os.environ["SPARK_HOME"]

sc = SparkContext("spark://129.114.82.28:7077", "Pilot-Spark KMeans")

from pyspark.mllib.clustering import KMeans, KMeansModel

SPARK HOME: /home1/01131/tg804093/src/supercomputing2015-tutorial/02_hadoop_on_hpc/work/spark-1.4.1-bin-hadoop2.6


## MLLib KMeans

In [3]:
# Load and parse the data
data = sc.textFile("data_1000points.csv")
parsedData = data.map(lambda line: array([float(x) for x in line.split(',')]))

# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 2, maxIterations=10,
        runs=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))



Within Set Sum of Squared Error = 9949.30979566


In [5]:
str(clusters)

'<pyspark.mllib.clustering.KMeansModel object at 0x2b03471b3810>'

In [ ]:
# Save and load model
clusters.save(sc, "kmeans.model")
sameModel = KMeansModel.load(sc, "kmeans.model")